In [25]:
import numpy as np
from SoftmaxRegression import SoftmaxRegression
import copy
import util

## 1. Data Loading

In [26]:
num_classes =  10 
validation_data = util.load_data("./data/classification_validation_data.pkl")
validation_label = util.load_data("./data/classification_validation_label.pkl")

In [27]:
# remove index column
train_data_index = validation_data[:, 0]
train_label_index = validation_label[:, 0]
validation_data = validation_data[:, 1:]
validation_label = validation_label[:, 1:].reshape(-1)

In [28]:
validation_data.shape, validation_label.shape

((9996, 256), (9996,))

In [29]:
train_data_index.shape, train_label_index.shape

((9996,), (9996,))

In [30]:

data= "./data/classification_validation_data.pkl"
label = "./data/classification_validation_label.pkl"
validation_data = util.load_data(data)
validation_label = util.load_data(label)
validation_data = validation_data[:, 1:]
validation_label = validation_label[:, 1:].reshape(-1)
validation_label = np.eye(num_classes)[validation_label]

linear_model = SoftmaxRegression(
        num_classes=num_classes,
        learning_rate=0.1,
        num_iterations=10000,
        random_seed=123)
        # no training, directly fix the weights 

linear_model.weights = util.load_data('./data/image_recognition_model_weights.pkl')
import random
class IEM:
    def __init__(self, mask, k=30) -> None:
        self.mask = mask
        self.k = k
        
    @classmethod
    def init_solution(self):
        select = set()
        while len(select) < 30:
            select.add(random.randint(0,255))
        mask_code = np.zeros((1, validation_data.shape[1]))
        for i in select:
            mask_code[0, i] = 1
        return IEM(mask_code)

    def value(self) -> int:
        mask_validation_data = validation_data * self.mask
        val_pred = linear_model.predict(mask_validation_data)
        val_accuracy = np.mean(val_pred == np.argmax(validation_label, axis=1))
        return val_accuracy
     
    def local_search(self):
        mask = copy.copy(self.mask)
        A = random.randint(0,255)
        B = random.randint(0,255)
        C = mask[0, A]
        mask[0, A] = mask[0, B]
        mask[0, B] = C
        iem = IEM(mask)
        return iem
    

    def save(self):
        util.save_data("./mask_code.pkl", self.mask)


def prob(a:float):
    return random.random() < a


def SA(initial:IEM, schedule, halt, log_interval=1, patient=20, spacial=0.1, ratio=0.1):
    state = initial.init_solution()
    t = 1
    T = schedule(t)
    f = []
    old_value = state.value()
    cnt = 0
    best = 0
    best_score = 0
    while not halt(T):
        now_value = old_value
        new_state = state.local_search()
        new_value = new_state.value()
        interest = np.exp(-(old_value - new_value)/T*ratio)
        interest = min(interest, 0.5)
        good = new_value >= old_value
        bad_but = (not good) and prob(interest)
        if good or bad_but:
            state = new_state
            old_value = new_value
            now_value = new_value
            cnt = 0
        else:
            cnt+=1
        if new_value > best_score:
            best_score = new_value
            best = new_state
        f.append(now_value)
        if t % log_interval == 0:
            if bad_but:
                print(f"step {t}: T={T}, current_value={now_value}")
            else:
                print(f"step {t}: T={T}, current_value={now_value}")
        t += 1
        T = schedule(T)
        if cnt >= patient:
            break
    
    return best, f



## 2. Feature Selection

In [31]:
iem = IEM.init_solution()
solution, record = SA(
    initial= iem,
    schedule=lambda t: 0.995*t, 
    halt=lambda T: T<1e-4,
    ratio = 0.1,
    patient=500,
    spacial=0.2
)
solution

step 1: T=0.995, current_value=0.14695878351340536
step 2: T=0.990025, current_value=0.14695878351340536
step 3: T=0.985074875, current_value=0.14695878351340536
step 4: T=0.9801495006250001, current_value=0.14695878351340536
step 5: T=0.9752487531218751, current_value=0.14695878351340536
step 6: T=0.9703725093562657, current_value=0.14695878351340536
step 7: T=0.9655206468094844, current_value=0.14695878351340536
step 8: T=0.960693043575437, current_value=0.14695878351340536
step 9: T=0.9558895783575597, current_value=0.14695878351340536
step 10: T=0.9511101304657719, current_value=0.14695878351340536
step 11: T=0.946354579813443, current_value=0.14695878351340536
step 12: T=0.9416228069143757, current_value=0.14695878351340536
step 13: T=0.9369146928798039, current_value=0.14695878351340536
step 14: T=0.9322301194154049, current_value=0.14695878351340536
step 15: T=0.9275689688183278, current_value=0.14695878351340536
step 16: T=0.9229311239742362, current_value=0.14695878351340536
s

In [32]:
solution.save()
print(solution.value())

Saved successfully
0.32262905162064826
